From: https://w.wiki/6aKG

In [1]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

from PIL import Image
import requests

endpoint_url = "https://query.wikidata.org/sparql"

query = """#defaultView:ImageGrid
SELECT ?item ?image ?Urheberrechtsstatus ?UrheberrechtsstatusLabel WHERE {
  ?item wdt:P31 wd:Q3305213.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  OPTIONAL { ?item wdt:P18 ?image. }
  OPTIONAL { ?item wdt:P6216 ?Urheberrechtsstatus. }
}
LIMIT 15"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    print(result)


{'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q609576'}, 'image': {'type': 'uri', 'value': 'http://commons.wikimedia.org/wiki/Special:FilePath/Anthonis%20van%20Dyck%20004.jpg'}, 'Urheberrechtsstatus': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q19652'}, 'UrheberrechtsstatusLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'public domain'}}
{'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q609576'}, 'image': {'type': 'uri', 'value': 'http://commons.wikimedia.org/wiki/Special:FilePath/Anthony%20van%20Dyck%20-%20Crowning%20with%20Thorns%20-%20WGA07433.jpg'}, 'Urheberrechtsstatus': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q19652'}, 'UrheberrechtsstatusLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'public domain'}}
{'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q609986'}, 'image': {'type': 'uri', 'value': 'http://commons.wikimedia.org/wiki/Special:FilePath/Retrat%20d%27Elisa%20Casas.jpg'}, 'U